In [10]:
import pandas as pd
import os
import json
import numpy as np


In [3]:
geodata_file = open('tractData.geojson')
geodata_str = geodata_file.read()
geo_data = json.loads(geodata_str)


In [50]:
# geo_data = pd.read_json(geodata_file)

# geo_data["features"][10]["properties"]["AFFGEOID"]
# geo_data["features"][0]['properties']
geo_data

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'geometry': {'bbox': [-120.76440199999999,
     38.130044999999996,
     -120.33448899999999,
     38.341100000000004],
    'type': 'Polygon',
    'coordinates': [[[-120.763991, 38.213891],
      [-120.761968, 38.217527000000004],
      [-120.762103, 38.22059],
      [-120.76138399999999, 38.223441],
      [-120.76242500000002, 38.226507999999995],
      [-120.76287400000001, 38.230942],
      [-120.76040900000001, 38.233346],
      [-120.76015999999998, 38.237083999999996],
      [-120.76043599999998, 38.24088],
      [-120.759672, 38.243027999999995],
      [-120.755645, 38.247713],
      [-120.754687, 38.25168],
      [-120.750684, 38.254942],
      [-120.748088, 38.255933999999996],
      [-120.74547, 38.256315],
      [-120.743996, 38.259691],
      [-120.752348, 38.261075],
      [-120.75786299999999, 38.258525999999996],
      [-120.76005500000001, 38.258168999999995],
      [-120.76307800000001, 38.2594200000000

In [15]:
# Read in csv file with population characteristic data by Sacramento County census tract
sac_data_file = 'sacRegionData.csv'
sac_data = pd.read_csv(sac_data_file).dropna()
sac_data.fillna(.0001)
sac_data.columns = ['GEO_ID', 'TRACT_NAME', 'BLACK', 'LATINO', 'LESS_NINTH', 'LESS_HS'] 
len(sac_data)
sac_data.head()

,GEO_ID,TRACT_NAME,BLACK,LATINO,LESS_NINTH,LESS_HS
0,1400000US06067000100,"Census Tract 1, Sacramento County, California",9.9,0.7,0.6,1
1,1400000US06067000200,"Census Tract 2, Sacramento County, California",14.9,0.1,0.9,0.7
2,1400000US06067000300,"Census Tract 3, Sacramento County, California",12.9,1.3,0.9,0.3
3,1400000US06067000400,"Census Tract 4, Sacramento County, California",18.2,8.9,1.6,3.1
4,1400000US06067000500,"Census Tract 5, Sacramento County, California",26.6,7.8,5.1,4.3


In [41]:
# This code loops through the geo json data and csv data;
# first checks for county code value, then adds new feature/
# properties if don't already exist

for x in range(len(geo_data["features"])):
    countyFP = geo_data["features"][x]["properties"]["COUNTYFP"]
    if(countyFP != "067"):
        continue

    geo_id = geo_data["features"][x]["properties"]["AFFGEOID"]
    for index, row in tract_data.iterrows():
        geo_df = row['GEO_ID']
        geo_NAME = row['TRACT_NAME']
        geo_DP05_0071PE = row['BLACK']
        geo_DP05_0038PE = row['LATINO']
        geo_S1501_C02_008E = row['LESS_NINTH']
        geo_S1501_C02_007E = row['LESS_HS']
        if(geo_id == geo_df):
            geo_data["features"][x]["properties"]["TRACT_NAME"] = geo_NAME
            geo_data["features"][x]["properties"]["BLACK"] = geo_DP05_0071PE
            geo_data["features"][x]["properties"]["LATINO"] = geo_DP05_0038PE
            geo_data["features"][x]["properties"]["LESS_NINTH"] = geo_S1501_C02_008E
            geo_data["features"][x]["properties"]["LESS_HS"] = geo_S1501_C02_007E

In [43]:
# dump JSON object with new properties into new JSON 
sac_dumps = json.dumps(geo_data)
open("sac2.json","w").write(sac_dumps)

13341888

In [33]:
## this is the full set of population data for all CA census tract;
# not using it here; too large!
tract_data_file = 'census_combined.csv'
tract_data = pd.read_csv(tract_data_file).dropna()
tract_data.columns = ['GEO_ID', 'TRACT_NAME', 'BLACK', 'LATINO', 'LESS_NINTH', 'LESS_HS'] 
len(tract_data)

8057

In [ ]:
# testing ability to access data
# tract_data['DP05_0071PE'].max()
# tract_data['DP05_0071PE'].min()

# for index, row in tract_data.iterrows():
#     print(row['GEO_ID'], row['DP05_0071PE'])

In [ ]:
## this code is for merging JSON GEOID and CSV population data 
## and output CSV with merged data to create CSV file same length
## as JSON data; not used

# build a list of geoids to merge with csv data?
affgeoid = []

for x in range(len(geo_data["features"])):
    affgeoid.append(geo_data["features"][x]['properties']["AFFGEOID"])
affgeoid

affgeoid_df = pd.DataFrame(affgeoid).dropna()
affgeoid_df.columns = ['GEO_ID']
affgeoid_df.head()

geoid_merge = pd.merge(affgeoid_df, tract_data, how='inner', on='GEO_ID')

geoid_merge.columns = ['GEO_ID', 'TRACT_NAME', 'BLACK', 'LATINO', 'LESS_NINTH', 'LESS_HS']
geoid_merge.fillna(0)

geoid_merge.to_csv("geoid_merge", encoding='utf-8', index=False)
